# 🔁 02 — Compare Sources (CBF vs Football-Data)

Objetivo:
- Comparar as fontes **CBF (Base dos Dados)** e **football-data.co.uk** no período comum (2012–2023)
- Normalizar nomes de times (canonização)
- Fazer matching por (data, mandante, visitante) com tolerância de **±1 dia**
- Gerar relatório de consistência e cobertura
- Exportar uma base “gold” padronizada (para treino/avaliação)

Saídas:
- `data/processed/gold_2012_2023.parquet`
- `reports/compare_sources_summary.md`

In [15]:
from google.colab import drive
drive.mount("/content/drive")

import pandas as pd
import numpy as np
from pathlib import Path

PROJECT = Path("/content/drive/MyDrive/DataProjects/BRMP-Brazilian-Match-Prediction")
INTERIM = PROJECT / "data" / "interim"
PROCESSED = PROJECT / "data" / "processed"
REPORTS = PROJECT / "reports"

PROCESSED.mkdir(parents=True, exist_ok=True)
REPORTS.mkdir(parents=True, exist_ok=True)

INTERIM, PROCESSED


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


(PosixPath('/content/drive/MyDrive/DataProjects/BRMP-Brazilian-Match-Prediction/data/interim'),
 PosixPath('/content/drive/MyDrive/DataProjects/BRMP-Brazilian-Match-Prediction/data/processed'))

In [16]:
cbf = pd.read_parquet(INTERIM / "cbf_2003_2023.parquet").copy()
fd  = pd.read_parquet(INTERIM / "fd_2012_2023.parquet").copy()

# recorte comum
cbf = cbf[(cbf["ano_campeonato"] >= 2012) & (cbf["ano_campeonato"] <= 2023)].copy()
fd  = fd[(fd["ano_campeonato"] >= 2012) & (fd["ano_campeonato"] <= 2023)].copy()

cbf.shape, fd.shape


((4559, 8), (4559, 7))

In [17]:
import sys
sys.path.insert(0, str(PROJECT))

from src.utils.team_aliases import canon_team

for df_ in (cbf, fd):
    df_["home_norm"] = df_["time_mandante"].map(canon_team)
    df_["away_norm"] = df_["time_visitante"].map(canon_team)

print("Nulos após canon (CBF):", cbf["home_norm"].isna().sum(), cbf["away_norm"].isna().sum())
print("Nulos após canon (FD) :", fd["home_norm"].isna().sum(), fd["away_norm"].isna().sum())


Nulos após canon (CBF): 0 0
Nulos após canon (FD) : 0 0


In [18]:
for df_ in (cbf, fd):
    df_["date_only"] = pd.to_datetime(df_["data"]).dt.date

cbf["match_key"] = cbf["date_only"].astype(str) + "|" + cbf["home_norm"] + "|" + cbf["away_norm"]
fd["match_key"]  = fd["date_only"].astype(str)  + "|" + fd["home_norm"]  + "|" + fd["away_norm"]

cbf["match_key"].nunique(), fd["match_key"].nunique()


(4559, 4559)

In [19]:
fd_keys = set(fd["match_key"])

def shift_key(df_, days):
    shifted = pd.to_datetime(df_["date_only"]) + pd.Timedelta(days=days)
    return shifted.dt.date.astype(str) + "|" + df_["home_norm"] + "|" + df_["away_norm"]

cbf["key_exact"] = cbf["match_key"]
cbf["key_m1"] = shift_key(cbf, -1)
cbf["key_p1"] = shift_key(cbf, +1)

match_exact = cbf["key_exact"].isin(fd_keys)
match_m1 = (~match_exact) & cbf["key_m1"].isin(fd_keys)
match_p1 = (~match_exact) & (~match_m1) & cbf["key_p1"].isin(fd_keys)

print("Match exato:", int(match_exact.sum()), "| cobertura:", round(match_exact.mean(), 4))
print("Match FD -1 dia:", int(match_m1.sum()), "| cobertura:", round(match_m1.mean(), 4))
print("Match FD +1 dia:", int(match_p1.sum()), "| cobertura:", round(match_p1.mean(), 4))
print("Match total (exato ou ±1):", int((match_exact|match_m1|match_p1).sum()), "| cobertura:", round((match_exact|match_m1|match_p1).mean(), 4))


Match exato: 3644 | cobertura: 0.7993
Match FD -1 dia: 780 | cobertura: 0.1711
Match FD +1 dia: 127 | cobertura: 0.0279
Match total (exato ou ±1): 4551 | cobertura: 0.9982


In [20]:
cbf["match_key_final"] = np.select(
    [match_exact, match_m1, match_p1],
    [cbf["key_exact"], cbf["key_m1"], cbf["key_p1"]],
    default=None
)

cbf["match_key_final"].isna().mean()


np.float64(0.001754770783066462)

In [21]:
fd_small = fd[["match_key", "gols_mandante", "gols_visitante", "resultado"]].copy()
fd_small = fd_small.rename(columns={
    "match_key": "match_key_final",
    "gols_mandante": "fd_gm",
    "gols_visitante": "fd_gv",
    "resultado": "fd_res"
})

merged = cbf.merge(fd_small, on="match_key_final", how="left")

print("Linhas:", merged.shape)
print("Sem match:", merged["fd_res"].isna().sum())

# checar consistência onde tem match
mask = merged["fd_res"].notna()
placar_ok = (merged.loc[mask, "gols_mandante"].values == merged.loc[mask, "fd_gm"].values) & \
            (merged.loc[mask, "gols_visitante"].values == merged.loc[mask, "fd_gv"].values)

print("Placares iguais:", int(placar_ok.sum()))
print("Placares diferentes:", int((~placar_ok).sum()))


Linhas: (4559, 19)
Sem match: 8
Placares iguais: 4551
Placares diferentes: 0


In [23]:
fd_std = fd.copy()

fd_std["data"] = pd.to_datetime(fd_std["data"])
fd_std["ano_campeonato"] = fd_std["ano_campeonato"]

fd_std["time_mandante"] = fd_std["time_mandante"]
fd_std["time_visitante"] = fd_std["time_visitante"]

fd_std["gols_mandante"] = fd_std["gols_mandante"]
fd_std["gols_visitante"] = fd_std["gols_visitante"]

fd_std["resultado"] = fd_std["resultado"]


In [24]:
gold = fd_std[[
    "data",
    "ano_campeonato",
    "time_mandante",
    "time_visitante",
    "gols_mandante",
    "gols_visitante",
    "resultado",
    "home_norm",
    "away_norm"
]].copy()

gold.shape


(4559, 9)

In [25]:
out = PROCESSED / "gold_2012_2023.parquet"
gold.to_parquet(out, index=False)

print("✅ gold exportado:", out)


✅ gold exportado: /content/drive/MyDrive/DataProjects/BRMP-Brazilian-Match-Prediction/data/processed/gold_2012_2023.parquet


## 📦 Gold Dataset — Schema & Padronização

O projeto utiliza um **Gold Dataset** padronizado como base única para treino, validação e simulação do modelo de Machine Learning.  
Esse dataset é derivado da fonte **football-data.co.uk**, após normalização de nomes de times e validações de consistência com a base da CBF.

**Arquivo:**
data/processed/gold_2012_2023.parquet


---

### 📐 Estrutura das Colunas

| Coluna            | Tipo       | Descrição |
|------------------|------------|-----------|
| `data`            | datetime   | Data oficial da partida |
| `ano_campeonato`  | int        | Ano da temporada |
| `time_mandante`   | string     | Nome original do time mandante |
| `time_visitante`  | string     | Nome original do time visitante |
| `home_norm`       | string     | Nome canonizado do mandante |
| `away_norm`       | string     | Nome canonizado do visitante |
| `gols_mandante`   | int        | Gols marcados pelo mandante |
| `gols_visitante`  | int        | Gols marcados pelo visitante |
| `resultado`       | category   | Resultado da partida (`H`, `D`, `A`) |

---

### 🎯 Target (`resultado`)

O target do modelo é a variável categórica:

- `H` → Vitória do mandante (Home Win)
- `D` → Empate (Draw)
- `A` → Vitória do visitante (Away Win)

Essa definição segue o padrão internacional adotado por casas de apostas e datasets esportivos.

---

### 🔄 Normalização de Times (Canonização)

Os nomes dos clubes são padronizados via um dicionário centralizado  
(`src/utils/team_aliases.py`) para garantir:

- Consistência entre diferentes fontes (CBF vs football-data)
- Evitar duplicações semânticas (ex: *Flamengo* vs *Flamengo RJ*)
- Estabilidade na engenharia de features históricas

**Exemplos:**
- `Flamengo RJ` → `flamengo`
- `Athletico-PR` → `athletico paranaense`
- `Atlético-MG` → `atletico mineiro`

---

### ⚠️ Observações Importantes

- A coluna `rodada` **não está presente** no Gold Dataset, pois não é fornecida pela fonte football-data.
- Métricas avançadas (odds, features rolling, força ofensiva/defensiva) são **derivadas posteriormente** durante a etapa de Feature Engineering.
- Dados brutos (`data/raw` e `data/interim`) **não são versionados** no repositório por boas práticas de engenharia de dados.

---

### ✅ Uso no Pipeline

O Gold Dataset é utilizado como entrada para:
- Engenharia de features
- Treinamento do modelo de classificação
- Avaliação (Log Loss, Brier Score)
- Simulação de temporadas futuras (ex: 2026)
